# Project 3 Web APIs & Classification

## Data Collection

In [1]:
# Import libraries
import requests
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm


%matplotlib inline
sns.set(font_scale=1.5)

#### 1. Exploring the data structure and identify the useful and relevanet data to the project on Reddit

In [2]:
#define the url
url="https://www.reddit.com/r/PS5/new.json"

In [3]:
#set the User-agent
header={"User-agent":"Daiyu52177475885"}

In [4]:
#access the address
res= requests.get(url, headers=header)

In [5]:
#get the request status
res.status_code

200

In [6]:
#check the keys in json file
data=res.json()
data.keys()

dict_keys(['kind', 'data'])

In [7]:
data["data"].keys()

dict_keys(['modhash', 'dist', 'children', 'after', 'before'])

In [8]:
data["data"]["children"][1]["data"]

{'approved_at_utc': None,
 'subreddit': 'PS5',
 'selftext': '',
 'author_fullname': 't2_563ahk4e',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': '[Video] Override 2: Super Mech League - Gameplay Trailer | PS4, PS5',
 'link_flair_richtext': [{'a': ':ps:',
   'e': 'emoji',
   'u': 'https://emoji.redditmedia.com/jcd8bjr024m41_t5_2s887/ps'},
  {'e': 'text', 't': ' Official'}],
 'subreddit_name_prefixed': 'r/PS5',
 'hidden': False,
 'pwls': 6,
 'link_flair_css_class': None,
 'downs': 0,
 'thumbnail_height': 105,
 'top_awarded_type': None,
 'hide_score': True,
 'name': 't3_j38ex8',
 'quarantine': False,
 'link_flair_text_color': 'light',
 'upvote_ratio': 0.8,
 'author_flair_background_color': None,
 'subreddit_type': 'public',
 'ups': 6,
 'total_awards_received': 0,
 'media_embed': {'content': '&lt;iframe width="600" height="338" src="https://www.youtube.com/embed/eBlRTsiFy3E?feature=oembed&amp;enablejsapi=1" frameborder="0" allow="accelerometer; auto

#### 2. Scrape Data and save it 

In [9]:
# Define function to scrape data
def scrape(urls, run):
   
    # Get posts as list of dictionaries, each containing data on one post
    posts = []
    for url in tqdm(urls):
        after = None
        for i in tqdm(range(run)):
            if after == None:
                params = {}
            else:
                params = {'after': after}
            headers = {'User-agent': 'GA SG Bot 0.1'}
            res = requests.get(url, params=params, headers=headers)
            if res.status_code == 200:
                the_json = res.json()
                posts.extend(the_json['data']['children'])
                after = the_json['data']['after']
            else:
                print(res.status_code)
                break
            time.sleep(1)  
    
    # Extract title, text and subreddit from data and return in a DataFrame
    titles, selftexts, subreddits, num_comments,upvote = [], [],[],[],[]
    for i in range(len(posts)):
        titles.append(posts[i]['data']['title'])
        selftexts.append(posts[i]['data']['selftext'])
        subreddits.append(posts[i]['data']['subreddit'])
        num_comments.append(posts[i]['data']['num_comments'])
        upvote.append(posts[i]['data']['upvote_ratio'])
    return pd.DataFrame({"title": titles,'selftext': selftexts, 'subreddit': subreddits,"num_comments": num_comments,\
                         "upvote":upvote})

In [10]:
# Scrape posts from subreddits
df = scrape(["https://www.reddit.com/r/XboxSeriesX.json?t=all","https://www.reddit.com/r/PS5/new/.json"], 50)

<ipython-input-9-75bb444f4b80>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm(urls):


<ipython-input-9-75bb444f4b80>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(run)):


In [11]:
#check the shape of the dataframe
df.shape

(2482, 5)

In [12]:
#do not run below code that will change the analysis results due to the updated posting
# Save results from scrape as csv
#df.to_csv('../data/game.csv', index=False)